In [1]:
import torch

In [2]:
from models import FiD
from utils import get_tokenizer
class args:
    def __init__(self):
        self.model_path = "/home/chi/code/data/BartBase"
        self.checkpoint = "../checkpoint/0419.ckpt"
        self.tokenizer = get_tokenizer(self.model_path, ["[ref]", "[title]", "[context]"])
        self.max_encoding_length = 256
        self.max_decoding_length =64
        self.loss_strategy = "alpha"

In [3]:
args = args()

In [4]:
model = FiD(args)
model.cuda()

loaded from ../checkpoint/0419.ckpt...


FiD(
  (bart): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50268, 768)
      (encoder): EncoderWrapper(
        (encoder): BartEncoder(
          (embed_tokens): Embedding(50268, 768)
          (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
          (layers): ModuleList(
            (0): BartEncoderLayer(
              (self_attn): BartAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
                (q_proj): Linear(in_features=768, out_features=768, bias=True)
                (out_proj): Linear(in_features=768, out_features=768, bias=True)
              )
              (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
              (activation_fn): GELUActivation()
              (fc1): Linear(in_features=768, out_features=3072, bias=True)
              (fc2): Linear(in_features=3072, out_f

In [5]:
import json
data = json.load(open("/home/chi/1111_full_res/regen_0301.json"))

In [ ]:
def 

In [6]:
data = [[k, "", data[k][1], i] for i,k in enumerate(data)]

In [113]:
test_data = copy.deepcopy(data[:4])
for i in range(len(test_data)):
    test_data[i][2].insert(0, "")
    n_ref = len(test_data[i][2])
    
    test_data[i].insert(-1, [0]*n_ref)

In [26]:
def generate_sentence(model, batch, data, topk=1):
    reference_scores, ctx_score, _ = model.get_scores(batch, False)
    
    print(ctx_score)

In [ ]:
generate_sentence()

In [110]:
import torch
from collections import defaultdict
from utils import InferDataset, vector_gather
from tqdm import tqdm
import copy
from torch.utils.data import DataLoader

def generate_sentence(model, batch, data, topk=1):
    reference_scores, ctx_score, _ = model.get_scores(batch, False)

    for i,line in enumerate(data):
        for n, count in enumerate(line[-2]):
            ctx_score[i][n] /=  (2**count)

    scores = reference_scores + ctx_score
    
    indices = ctx_score.topk(topk, dim=-1)[1]
    for i,idx in enumerate(indices):
#         print(f"第{i}个样本本次使用:{idx.item()}")
        data[i][-2][idx.item()] += 1
    ctx_input_ids = batch["input_ids"][:,0,:].unsqueeze(1)
    ctx_attention_mask = batch["attention_mask"][:,0,:].unsqueeze(1)
    
    ref_input_ids = batch["input_ids"][:,1:,:]
    ref_attention_mask = batch["attention_mask"][:,1:,:]
    
    batch["input_ids"] = vector_gather(ref_input_ids, indices)
    batch["input_ids"] = torch.cat([ctx_input_ids, batch["input_ids"]], dim=1)
    
    batch["attention_mask"] = vector_gather(ref_attention_mask, indices)
    batch["attention_mask"] = torch.cat([ctx_attention_mask, batch["attention_mask"]], dim=1)
    
    sents = model.generate(batch)
    
    for i,idx in enumerate(indices):
        if reference_scores[i][idx.item()] <= reference_scores[i][0]:
            sents[i] = ""
        
#     for i,s in enumerate(reference_scores):
#         for k,p in enumerate(s):
#             ref_p = p.item()
#             ctx_p = ctx_score[i][k].item()
#             if data[i][2][k]:
#                 print(f"第{i}个样本第{k}个reference score的得分是:{round(ref_p,4)}; ctx score的得分是:{round(ctx_p,4)}")
#             else:
#                 print(f"第{i}个样本第{k}个reference score的得分是:{round(ref_p,4)}; ctx score的得分是:{round(ctx_p,4)}, 此样本已被干")
#         print("="*100)
#         print(f"第{i}个样本本次生成: {sents[i]}")
#         print(f"第{i}个样本已经生成: {data[i][1]}")
#         print(f"第{i}个样本本次使用: {data[i][2][indices[i].item()]}")
#         print("="*100)
    
    return sents, indices.tolist()

def generate_by_sentence(data, num_sent=3, batch_size=4):
    record = defaultdict(list)
    for i in range(num_sent):
        res_dict = {}
        dl = InferDataset(data, args)
        DL = DataLoader(dl,
                    batch_size=batch_size)
        for batch in tqdm(DL):
            ctx, used_indices = generate_sentence(model, batch, data)
            for k,idx in enumerate(batch["id"]):
                idx = idx.item()
                res_dict[idx] = [ctx[k], used_indices[k]]
                record[idx].append([ctx[k], used_indices[k]])
        data = arrange_data(res_dict, data)
    return data, record

# def write_res(record, out_path):
#     with open(out_path, "w") as f:
#         for i in range(len(record)):
#             tmp = []
#             for sent in record[i]:
#                 ref_ids = "".join([f"[{i+1}]" for i in sent[1]])
#                 tmp.append(f"{sent[0]} {ref_ids}")
#             f.write(" ".join(tmp)+"\n")

def arrange_data(res_dict, last_data):
    rtn = []
    for i,line in enumerate(last_data):
        if line[1]:
            line[1] = f"{line[1]} {res_dict[i][0]}"
        else:
            line[1] = res_dict[i][0]
#         for idx in res_dict[i][1]:
#             line[2][idx] = ""
        rtn.append(line)
    return rtn

In [65]:
from nlgeval import NLGEval
nlgeval = NLGEval(no_skipthoughts=True, no_glove=True)  # loads the models

In [114]:
# test_data = copy.deepcopy(data[:2])
test_data, record = generate_by_sentence(test_data, num_sent=5, batch_size=4)

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 84.00 MiB (GPU 0; 15.90 GiB total capacity; 6.63 GiB already allocated; 60.38 MiB free; 6.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [106]:
def write_res(record):
    res = []
    for i in range(len(record)):
        tmp = []
        for sent in record[i]:
#             if sent[0] in tmp: continue
            tmp.append(f"{sent[0]}")
        tmp_sent = " ".join(tmp)
        tmp_sent = re.sub("\s+", " ", tmp_sent)
        res.append(tmp_sent.lower())
    return res

In [107]:
hypos = write_res(record)
metrics_dict = nlgeval.compute_metrics(gold, hypos)
for key in metrics_dict:
    print(f"{key}: {metrics_dict[key]}")



Bleu_1: 0.18224468952600945
Bleu_2: 0.09470079484504816
Bleu_3: 0.044712507637329486
Bleu_4: 3.875642588889751e-06
METEOR: 0.09118862637318977
ROUGE_L: 0.1528810542880321
CIDEr: 0.005113366431401963


In [21]:
hypos = write_res(record)
metrics_dict = nlgeval.compute_metrics(gold, hypos)
for key in metrics_dict:
    print(f"{key}: {metrics_dict[key]}")

Bleu_1: 0.12771325996116456
Bleu_2: 0.05296893601371125
Bleu_3: 0.019073389459848412
Bleu_4: 2.042063640415763e-06
METEOR: 0.08966202017457767
ROUGE_L: 0.1329529646591036
CIDEr: 0.027922167373311598


In [67]:
import re
gold = open("/home/chi/webbrain2/output/gold_100.res").readlines()[:4]
gold = [re.sub(r"(\[[0-9]+\])", r"", s) for s in gold]
gold = [[s.strip() for s in gold]]


In [34]:
hypos = write_res(record)
metrics_dict = nlgeval.compute_metrics(gold, hypos)
for key in metrics_dict:
    print(f"{key}: {metrics_dict[key]}")

Bleu_1: 0.15318606568414836
Bleu_2: 0.0705478713815973
Bleu_3: 0.03354619117201066
Bleu_4: 3.1348456395344754e-06
METEOR: 0.09305835332152121
ROUGE_L: 0.12682471380408974
CIDEr: 0.0007407013791504158
